In [1]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Урок 7. Сверточные нейронные сети для анализа текста

Задание
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации

In [4]:
import pandas as pd

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping
from tensorflow.keras import utils as np_utils

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
df = pd.read_excel('отзывы за лето.xls')
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [6]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

In [7]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df['prepared_Content'] = df['Content'].apply(preprocess_text)
df.head()

,Rating,Content,Date,prepared_Content
0,5,It just works!,2017-08-14,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...
2,5,Отлично все,2017-08-14,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро


In [8]:
train_corpus = " ".join(df['prepared_Content'])
train_corpus = train_corpus.lower()

In [9]:
tokens = word_tokenize(train_corpus)

In [10]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [11]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
#tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [12]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]


#x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df['prepared_Content'], df['Rating'], test_size=0.25, random_state=54)

In [14]:
from collections import Counter

words_counter = Counter((word for text in X_train for word in text.lower().split()))

word2idx = {
    '': 0,
    '<unk>': 1
}
for word, count in words_counter.most_common():
    if count < 10:
        break
        
    word2idx[word] = len(word2idx)
    
print('Words count', len(word2idx))

Words count 965


In [15]:
def convert(texts, word2idx, max_text_len):
    data = np.zeros((len(texts), max_text_len), dtype=np.int)
    
    for inx, text in enumerate(texts):
        result = []
        for word in text.split():
            if word in word2idx:
                result.append(word2idx[word])
        padding = [0]*(max_text_len - len(result))
        data[inx] = np.array(padding + result[-max_text_len:], dtype=np.int)
    return data

X_train = convert(X_train, word2idx, 1000)
X_test = convert(X_test, word2idx, 1000)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprec

In [16]:
y_train = y_train.apply (lambda x: x-1)
y_test = y_test.apply (lambda x: x-1)

In [17]:
num_classes = 5
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [18]:
model = Sequential()
model.add(Embedding(input_dim=len(word2idx), output_dim=128, input_shape=(X_train.shape[1],)))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 10
batch_size = 1024
print_batch_n = 100

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    #callbacks=[tensorboard, early_stopping],
                    validation_data=(X_test, y_test))

Epoch 1/10
14/14 [==============================] - 104s 7s/step - loss: 1.5033 - accuracy: 0.5952 - val_loss: 1.3085 - val_accuracy: 0.7194
Epoch 2/10
14/14 [==============================] - 99s 7s/step - loss: 1.1242 - accuracy: 0.7084 - val_loss: 0.9545 - val_accuracy: 0.7194
Epoch 3/10
14/14 [==============================] - 99s 7s/step - loss: 0.9551 - accuracy: 0.7084 - val_loss: 0.8904 - val_accuracy: 0.7194
Epoch 4/10
14/14 [==============================] - 100s 7s/step - loss: 0.8634 - accuracy: 0.7111 - val_loss: 0.8046 - val_accuracy: 0.7335
Epoch 5/10
14/14 [==============================] - 101s 7s/step - loss: 0.7689 - accuracy: 0.7447 - val_loss: 0.7345 - val_accuracy: 0.7542
Epoch 6/10
14/14 [==============================] - 102s 7s/step - loss: 0.7114 - accuracy: 0.7624 - val_loss: 0.6985 - val_accuracy: 0.7561
Epoch 7/10
14/14 [==============================] - 101s 7s/step - loss: 0.6830 - accuracy: 0.7689 - val_loss: 0.6841 - val_accuracy: 0.7548
Epoch 8/10
14/1

2. Рассмотреть 2-а варианта сеточек
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [21]:
path_to_glove_file = 'model.txt'
embeddings_index = {}
with open(path_to_glove_file, encoding='utf8') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 189194 word vectors.


In [22]:
num_tokens = len(word2idx)
embedding_dim = 300
hits = 0
misses = 0
misses_words = []

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for i, word in words_counter.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
        misses_words.append(word)

In [23]:
model_2 = Sequential()
model_2.add(Embedding(num_tokens,
            embedding_dim,
            embeddings_initializer=keras.initializers.Constant(embedding_matrix),
            trainable=False,
    ))
model_2.add(Conv1D(128, 3))
model_2.add(Activation("relu"))
model_2.add(GlobalMaxPool1D())
model_2.add(Dense(10))
model_2.add(Activation("relu"))
model_2.add(Dense(num_classes))
model_2.add(Activation('softmax'))

In [24]:
model_2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [25]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 10
batch_size = 1024
print_batch_n = 100

history = model_2.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    #callbacks=[tensorboard, early_stopping],
                    validation_data=(X_test, y_test))

Epoch 1/10
14/14 [==============================] - 159s 11s/step - loss: 1.6030 - accuracy: 0.6650 - val_loss: 1.5950 - val_accuracy: 0.7194
Epoch 2/10
14/14 [==============================] - 151s 11s/step - loss: 1.5889 - accuracy: 0.7084 - val_loss: 1.5807 - val_accuracy: 0.7194
Epoch 3/10
14/14 [==============================] - 152s 11s/step - loss: 1.5750 - accuracy: 0.7084 - val_loss: 1.5666 - val_accuracy: 0.7194
Epoch 4/10
14/14 [==============================] - 151s 11s/step - loss: 1.5613 - accuracy: 0.7084 - val_loss: 1.5526 - val_accuracy: 0.7194
Epoch 5/10
14/14 [==============================] - 152s 11s/step - loss: 1.5477 - accuracy: 0.7084 - val_loss: 1.5389 - val_accuracy: 0.7194
Epoch 6/10
14/14 [==============================] - 152s 11s/step - loss: 1.5344 - accuracy: 0.7084 - val_loss: 1.5253 - val_accuracy: 0.7194
Epoch 7/10
14/14 [==============================] - 154s 11s/step - loss: 1.5212 - accuracy: 0.7084 - val_loss: 1.5120 - val_accuracy: 0.7194
Epoch 

Сеть с предобученым энбединг слоем показал результат хуже чем сеть с обучающимся энбедингом и само обочение идёт хуже.